In [3]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
import os
import time
from typing import List, NamedTuple

import os
import time
from typing import List, NamedTuple

import matplotlib.pyplot as plt
import numpy as np
from IPython import display
from qiskit import QuantumCircuit
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.operators import PauliExpectation
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.operators.converters import AbelianGrouper
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.circuit import Parameter
from qiskit.providers.aer import QasmSimulator, AerProvider
from qiskit.providers.aer.noise import NoiseModel
from qiskit.quantum_info import Statevector
from qiskit.test.mock import *
from qiskit.visualization import plot_error_map

In [ ]:
class CallbackLogger:
    """
    Pass an instance of this as a callback to VQE.__init__, and access the log in self.vqe_log when the run is done.
    """

    def __init__(self, sim_model, molecule):
        if sim_model == "Statevector":
            self.vqe_log = LogIdealCircuit()
        elif sim_model == "NoisyCircuit":
            self.vqe_log = LogShotNoise()
        elif sim_model == "FakeVigo":
            self.vqe_log = LogFakeVigo()
        self.molecule = molecule

    def __call__(self, eval_count: int,
                 parameters: np.ndarray,
                 energy_mean: float,
                 energy_std: float) -> None:
        self.vqe_log.eval_count_list.append(eval_count)
        self.vqe_log.parameters_list.append(parameters)
        self.vqe_log.energy_mean_list.append(energy_mean)
        self.vqe_log.energy_std_list.append(energy_std)

        if eval_count % 2 == 0:
            plt.clf()
            plt.title(r"$H_2$ Energy Optimization using VQE")
            plt.xlabel("Iter")
            plt.ylabel("Mean Energy")
            plt.axvline(linestyle='__', color='k', alpha=0.5, label="Calibration Steps")
            mean_energy = (np.array(self.vqe_log.energy_mean_list[::2]) + np.array(
                self.vqe_log.energy_mean_list[1::2])) / 2
            plt.plot(np.arange(0, len(mean_energy)) - 25, mean_energy)
            plt.axhline(y=self.molecule.groundstate_energy.real + 0.0016, color='r', label='Chemical Accuracy')
            plt.axhline(y=self.molecule.groundstate_energy.real, color=' k', label=r'Exact $E_0$')
            plt.legend()
            display.display(plt.gcf())
            print(f"Optimization Iter {eval_count // 2 - 25} | Mean Energy = {energy_mean}")
            display.clear_output(wait=True)


class LogIdealCircuit(NamedTuple):
    """
    Container for a log of a VQE run on an ideal circuit. Constructed by VQELogger
    """
    eval_count_list: List[int] = []
    parameters_list: List[np.ndarray] = []
    energy_mean_list: List[float] = []
    energy_std_list: List[float] = []


class LogShotNoise(NamedTuple):
    """
    Container for a log of a VQE run with measurement outcomes sampled from noisy circuits. Constructed by VQELogger
    """
    eval_count_list: List[int] = []
    parameters_list: List[np.ndarray] = []
    energy_mean_list: List[float] = []
    energy_std_list: List[float] = []


class LogFakeVigo(NamedTuple):
    """
    Container for a log of a VQE run. Constructed by VQELogger
    """
    eval_count_list: List[int] = []
    parameters_list: List[np.ndarray] = []
    energy_mean_list: List[float] = []
    energy_std_list: List[float] = []

In [4]:
os.environ['QISKIT_IN_PARALLEL'] = 'TRUE'
plt.rcParams["figure.dpi"] = 75

In [20]:
file_path = './qmolecule-data'
bond_lengths = np.linspace(0.1, 4, 60)

In [18]:
def create_QMolecule(bond_length):
    driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(bond_length), unit=UnitsType.ANGSTROM, charge=0,
                             spin=0, basis='sto3g')
    #driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(bond_length), unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto6g')
    molecule = driver.run()
    file_name = '_'.join(['h2',str(bond_length).replace('.','_')])
    molecule.save(os.path.join(file_path,file_name))

In [21]:
for bond_length in bond_lengths:
    create_QMolecule(bond_length)